In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
aws_region = os.getenv("AWS_DEFAULT_REGION")
tavily_api_key = os.getenv("TAVILY_API_KEY")

### Tavily 도구 설정

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults
# from dabom_langchain import DabomSearchResults
# from pydantic import BaseModel
# Tavily 검색 도구 설정 (삼성전자 주가 검색용)
search = TavilySearchResults(max_results=2)
# search = DabomSearchResults(max_results=2)

# 샘플 검색 실행: 삼성전자 주가
search_results = search.invoke("삼성전자(005930) 주가")
print(search_results)

# 도구 목록에 추가
tools = [search]


[{'url': 'https://alphasquare.co.kr/home/stock-summary?code=005930', 'content': '삼성전자의 실시간 주가, 차트, 기업 정보, 매출 정보, 투자자별 매매동향 등을 확인할 수 있는 웹사이트입니다. 삼성전자의 주가 현재가는 64500원이며, 변동률은 -2.57%이다.'}, {'url': 'https://kr.investing.com/equities/samsung-electronics-co-ltd', 'content': '삼성전자의 현재 주가, 적정가치, 기술적 분석, 뉴스 및 분석, 재정 상황 등을 확인하세요. 삼성전자는 세계 최대의 전자제조업체로, 2024년 10월 30일에 다음 실적을 발표할 예정입니다.'}]


### 언어 모델 사용

In [10]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

# Claude 모델 설정
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

# 도구를 바인딩한 모델 생성
model_with_tools = model.bind_tools(tools)

# 삼성전자 주가에 대한 질문 처리
response = model_with_tools.invoke([HumanMessage(content="삼성전자 주가가 어떻게 되나요?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

### 딱히 어떤 정보 검색이 필요하지 않으면 ToolCalls가 비어서 나옴

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_019nsHsbfX5trvcBMLi7FCwn', 'type': 'tool_call'}]


### 에이전트 생성

In [11]:
from langgraph.prebuilt import create_react_agent

# 에이전트 생성
agent_executor = create_react_agent(model, tools)

# 삼성전자 주가 질문 처리
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="삼성전자 주가가 어떻게 되나요?")]}
).get("messages")
print(response)

[HumanMessage(content='삼성전자 주가가 어떻게 되나요?', additional_kwargs={}, response_metadata={}, id='7c5c3f73-328f-4c29-8e08-b6fe0015e66d'), AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 417, 'completion_tokens': 113, 'total_tokens': 530}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 417, 'completion_tokens': 113, 'total_tokens': 530}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-fb82540f-9b59-48b1-8577-b101193435b8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_013wE5wWUmsMchgKoLcWZHM2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 113, 'total_tokens': 530}), ToolMessage(content='[{"url": "https://kr.investing.com/equities/samsung-electronics-co-ltd", "content": "삼성전자 (005930) 삼성전자은/는 지난 해 (52주 동안) 거래된 가격 중 가장 낮은 가격을 기록했습니다. 이것은 주식의 현재 가치를 분석하고 향후 가격 움

In [12]:
response[-1].pretty_print()

================================== Ai Message ==================================

삼성전자의 현재 주가에 대해 검색해보았습니다. 실시간 주가 정보는 계속 변동되기 때문에 정확한 현재 가격을 말씀드리기는 어렵습니다. 하지만 몇 가지 중요한 정보를 공유해 드리겠습니다:

1. 삼성전자의 주식 코드는 005930입니다.

2. 최근 삼성전자 주가는 52주 동안의 최저점을 기록했다고 합니다. 이는 주식의 현재 가치를 분석하고 향후 가격 움직임을 예측하는 데 사용될 수 있는 중요한 기술적 지표입니다.

3. 2021년 말 기준으로 삼성전자 주가는 7만원대에서 마감되었습니다. 그 해에는 연간 수익률이 마이너스를 기록하며 부진한 모습을 보였습니다.

4. 2022년 전망으로는 메모리 반도체 업사이클에 힘입어 많은 증권사들이 삼성전자의 목표주가를 10만원에서 12만원으로 상향 조정했다고 합니다.

정확한 현재 주가를 알고 싶으시다면, 주식 거래 플랫폼이나 실시간 금융 정보 제공 웹사이트를 확인하시는 것이 가장 좋습니다. 주가는 시시각각 변동되므로, 실시간 데이터를 제공하는 신뢰할 수 있는 소스를 이용하시는 것이 중요합니다.


### 스트리밍 방식으로 응답 확인

In [15]:
# 스트리밍 메시지 실행
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="삼성전자 주가가 어떻게 되나요?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 417, 'completion_tokens': 148, 'total_tokens': 565}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 417, 'completion_tokens': 148, 'total_tokens': 565}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-69825341-e944-4c16-afba-6610f55a73e4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_01RawPtdRzcsPQFoxRHUjVfn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 148, 'total_tokens': 565})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://kr.investing.com/equities/samsung-electronics-co-ltd", "content": "삼성전자 (005930) 삼성전자은/는 지난 해 (52주 동안) 거래된 가격 중 가장 낮은 가격을 기록했습니다. 이것은 주식의 현재 가치를 분석하고 향후 가격 움직임을 예측하는 데 사용할 수 있는 기술적 지표입니다. Samsung Electronics Co Ltd 주가가 실시간인 경우 이 ..."

### 메모리 추가

In [20]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 설정
memory = MemorySaver()

# 메모리 적용된 에이전트 생성
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# 대화 스레드 ID 설정
config = {"configurable": {"thread_id": "abc123"}}

# 삼성전자 주가 질문 처리 후 메모리 확인
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="삼성전자 주가가 어떻게 되나요?")]}, config
):
    print(chunk)
    print("----")

# 이후 대화에서 이전 질문을 기억하는지 확인
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="내가 전에 뭐 물어봤지?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'usage': {'prompt_tokens': 417, 'completion_tokens': 110, 'total_tokens': 527}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 417, 'completion_tokens': 110, 'total_tokens': 527}, 'stop_reason': 'tool_use', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-37a582a6-3b73-43c2-8712-d5daf67f47df-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '삼성전자 현재 주가'}, 'id': 'toolu_bdrk_01CcXwsNKcvm6N8KNDXgKhUm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 417, 'output_tokens': 110, 'total_tokens': 527})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://kr.investing.com/equities/samsung-electronics-co-ltd", "content": "삼성전자 (005930) 삼성전자은/는 지난 해 (52주 동안) 거래된 가격 중 가장 낮은 가격을 기록했습니다. 이것은 주식의 현재 가치를 분석하고 향후 가격 움직임을 예측하는 데 사용할 수 있는 기술적 지표입니다. Samsung Electronics Co Ltd 주가가 실시간인 경우 이 ..."